In [1]:
import os
import numpy as np
import tensorflow as tf
# import tensorflow_datasets as tfds
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Load the IMDB Reviews dataset
imdb = pd.read_csv("../data/movie/IMDB.csv")

In [3]:
imdb.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
imdb['sentiment'] = imdb['sentiment'].map({'positive': 1, 'negative': 0})

In [5]:
imdb.shape

(50000, 2)

In [6]:
X = imdb["review"]
y = imdb["sentiment"]

In [7]:
training_sentences, testing_sentences, training_labels, testing_labels = train_test_split(X,y,test_size=0.5,random_state=42)

In [8]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

### Generate Padded Sequences

In [9]:
# Parameters
vocab_size = 10000
max_length = 120
embedding_dim = 16
trunc_type='post'
oov_tok = "<OOV>"

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# Generate the word index dictionary for the training sentences
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

# Generate and pad the training sequences
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

# Generate and pad the test sequences
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, truncating=trunc_type)

### Build and Compile the Model

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Setup the training parameters
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [12]:
#summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


### Train the Model

In [13]:
num_epochs = 10

# Train the model
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 4ms/step - loss: 0.4949 - accuracy: 0.7418 - val_loss: 0.3655 - val_accuracy: 0.8375
Epoch 2/10
782/782 [==============================] - 3s 3ms/step - loss: 0.2515 - accuracy: 0.9002 - val_loss: 0.3748 - val_accuracy: 0.8354
Epoch 3/10
782/782 [==============================] - 2s 3ms/step - loss: 0.1066 - accuracy: 0.9701 - val_loss: 0.4561 - val_accuracy: 0.8273
Epoch 4/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0265 - accuracy: 0.9965 - val_loss: 0.5532 - val_accuracy: 0.8223
Epoch 5/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0067 - accuracy: 0.9996 - val_loss: 0.6069 - val_accuracy: 0.8257
Epoch 6/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0022 - accuracy: 0.9999 - val_loss: 0.6646 - val_accuracy: 0.8257
Epoch 7/10
782/782 [==============================] - 2s 3ms/step - loss: 9.4885e-04 - accuracy: 1.0000 - val_loss: 0.7120 - val_accuracy: 0.8269
Ep

### Visualize Word Embeddings

In [14]:
# Get the embedding layer from the model (i.e. first layer)
embedding_layer = model.layers[0]

# Get the weights of the embedding layer
embedding_weights = embedding_layer.get_weights()[0]

# Print the shape. Expected is (vocab_size, embedding_dim)
print(embedding_weights.shape) 

(10000, 16)


In [15]:
# Get the index-word dictionary
reverse_word_index = tokenizer.index_word

In [16]:
import io

# Open writeable files
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

# Initialize the loop. Start counting at `1` because `0` is just for the padding
for word_num in range(1, vocab_size):

  # Get the word associated at the current index
  word_name = reverse_word_index[word_num]

  # Get the embedding weights associated with the current index
  word_embedding = embedding_weights[word_num]

  # Write the word name
  out_m.write(word_name + "\n")

  # Write the word embedding
  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

# Close the files
out_v.close()
out_m.close()